In [4]:
# https://github.com/RicardoMoya/Reinforcemente_Learning_with_Python/blob/master/4_02_RL_SpaceInvaders_GYM_KerasRL.ipynb
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Filtramos los warnings
import warnings
warnings.filterwarnings("ignore")

# Forzar uso CPU en caso de dispones GPU en el PC
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

Juego Space Invaders de Open AI GYM (con Keras RL)
GYM de Open AI https://gym.openai.com/: Gym es un conjunto de herramientas para desarrollar y comparar algoritmos de Aprendizaje por Refuerzo (RL). Esta librería ofrece diferentes juegos (entornos) como los juegos de Atari en los que poder probar los algoritmos de RL.

Keras RL https://keras-rl.readthedocs.io/: keras-rl implementa algunos algoritmos de Aprendizaje por Refuerzo en Python y se integra con con Tensorflow-Keras. Además, keras-rl esta listo para su uso con OpenAI Gym, por lo que resulta sencillo evaluar mediante los juegos de GYM el rendimiento de los Algoritmos.

In [ ]:
!pip install tensorflow==2.10.0
!pip install keras-rl2
!pip3 install atari_py --user
!pip install gym[atari,accept-rom-license]==0.23.0  # Reinstall with Atari support

In [10]:
# 1.- Instalación de las librerías
!pip install tensorflow==2.10.0
#!pip uninstall atari_py -y  # Uninstall any existing versions
!pip install atari_py  # Reinstall without --user
!pip install gym[atari]  # Reinstall gym with Atari dependencies

# 2.- Importamos la librería de GYM y atari
import gym
import atari_py  # Try importing again

# Definición de constantes
ENVIRONMENT_NAME = "SpaceInvaders-v0"

# 3.- Instanciamos el entorno del Space Invaders y jugamos con un agente que toma acciones aleatorias
# Instanciamos el Entorno
env = gym.make(ENVIRONMENT_NAME)

Found existing installation: atari-py 0.2.9
Uninstalling atari-py-0.2.9:
  Successfully uninstalled atari-py-0.2.9
  Using cached atari_py-0.2.9-cp310-cp310-linux_x86_64.whl


In [11]:
# Obtenemos el tamaño de las imagenes del juego y los canales
height, width, n_channels = env.observation_space.shape
print('Tamaño de la imagen: {}x{} - Nº Canales: {}'.format(width, height, n_channels))

# Vemos las posibles acciones del juego:
n_actions = env.action_space.n
actions_info = env.unwrapped.get_action_meanings()
print('\nNº de acciones: {} - Acciones: {}\n'.format(n_actions, actions_info))

# Jugamos 2 partidas de manera aleatoria
episodes = 2
for episode in range(1, episodes+1):
    state = env.reset()  # Inicializamos el entorno
    done = False         # Flag de finalización de la partida (episodio)
    score = 0            # Contador de recompensas

    # ! A Jugar ¡ (Hasta que termine la partida -> done == True)
    while not done:
        action = env.action_space.sample()              # Seleccionamos una acción Aleatoria
        n_state, reward, done, info = env.step(action)  # Realizamos la acción aleatoria y obtenemos:
                                                        # 1. Lista de estados, 2. Recompensa, 3. ¿Fin del juego?, 4. info
        score+=reward                                   # Sumamos la recompensa de la acción
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Tamaño de la imagen: 160x210 - Nº Canales: 3

Nº de acciones: 6 - Acciones: ['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

Episode:1 Score:105.0
Episode:2 Score:105.0


In [12]:
# 4.- Creamos un modelo de red neuronal convolucional
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

def build_cnn_model():
    model = Sequential()
    # The input_shape was corrected to (height, width, n_channels)
    model.add(Convolution2D(filters=32,
                            kernel_size=(8,8),
                            strides=(4,4),
                            activation='relu',
                            input_shape=(height, width, n_channels)))
    model.add(Convolution2D(filters=64,
                            kernel_size=(4,4),
                            strides=(2,2),
                            activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(n_actions, activation='linear'))

    return model

model = build_cnn_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 51, 39, 32)        6176      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 18, 64)        32832     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 512)               14156288  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 14,328,166
Trainable params: 14,328,166
No

In [14]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
import gym
import atari_py
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

# Definición de constantes
ENVIRONMENT_NAME = "SpaceInvaders-v0"

# Instanciamos el Entorno
env = gym.make(ENVIRONMENT_NAME)

# Obtenemos el tamaño de las imagenes del juego y los canales
height, width, n_channels = env.observation_space.shape
# Vemos las posibles acciones del juego:
n_actions = env.action_space.n

# Number of frames to stack
window_length = 4

def build_cnn_model():
    model = Sequential()
    # The input_shape now includes the window_length
    model.add(Convolution2D(filters=32,
                            kernel_size=(8,8),
                            strides=(4,4),
                            activation='relu',
                            input_shape=(window_length, height, width, n_channels)))
    model.add(Convolution2D(filters=64,
                            kernel_size=(4,4),
                            strides=(2,2),
                            activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(n_actions, activation='linear'))

    return model

model = build_cnn_model()
memory = SequentialMemory(limit=1000, window_length=window_length)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
dqn = DQNAgent(model=model,
               memory=memory,
               policy=policy,
               enable_dueling_network=True,
               nb_actions=n_actions,
               nb_steps_warmup=100)

# Compilamos el modelo
dqn.compile(Adam(lr=1e-4))

# Entrenamos el modelo
dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 34261s 3s/step - reward: 0.2070
done, took 34260.692 seconds


In [29]:
# 6.- Explotamos el modelo. Nos ponemos a jugar con el agente ya entrenado.
env_test = gym.make(ENVIRONMENT_NAME, render_mode='human')
scores = dqn.test(env_test, nb_episodes=5, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 5 episodes ...
Episode 1: reward: 105.000, steps: 662
Episode 2: reward: 335.000, steps: 1049
Episode 3: reward: 105.000, steps: 762
Episode 4: reward: 120.000, steps: 655
Episode 5: reward: 145.000, steps: 905
162.0
